In [2]:
from TinySAM import *
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm.auto import tqdm, trange
from torch.utils.data import DataLoader

ImportError: cannot import name '_cuda' from 'torch._utils' (/home/johnson/mambaforge/envs/TinySAM_/lib/python3.10/site-packages/torch/_utils.py)

# Setting Up Models

In [ ]:
GroundingModel = GDino()
SAMModel = EdgeSAM()

# Load Data to RAM

In [ ]:
data = ZeroShotObjectDetectionDataset('./Data/cityscapes/',do_preprocess=False, processor=GroundingModel.processor)

In [ ]:
# visualize a random sample
rnd_idx = np.random.randint(0,len(data))
data.visualize(rnd_idx)

# Run Grounding Dino

In [ ]:
batch_size = 8
dataloader = DataLoader(data, batch_size=batch_size, shuffle=False)

boxes = []
labels = []
scores = []

# get the text prompts
input_ids = data.input_prompt_ins.input_ids.to(GroundingModel.device)
target_image_size = data.image_size

for batch in tqdm(dataloader):
    outputs = GroundingModel(batch.to(GroundingModel.device), input_ids=input_ids, target_image_size=target_image_size)
    
    for out in outputs:
        boxes.append(out['boxes'].cpu().numpy())
        labels.append(out['labels'])
        scores.append(out['scores'].cpu().numpy())
        for i in range(len(labels[-1])):
            # only pick first label for any dual predictions
            if labels[-1][i] + '.' not in data.text_prompts:
                labels[-1][i] = labels[-1][i].split(' ')[0]

# Run SAM2

In [ ]:
masks = SAMModel(data.images, boxes)

In [ ]:
# Save Results (if needed)
np.save('masks.npy', np.array(masks, dtype=object))
np.save('boxes.npy', np.array(boxes, dtype=object))
np.save('labels.npy', np.array(labels, dtype=object))
np.save('scores.npy', np.array(scores, dtype=object))

In [ ]:
# Load Results (if needed)
masks = np.load('masks.npy', allow_pickle=True)
boxes = np.load('boxes.npy', allow_pickle=True)
labels = np.load('labels.npy', allow_pickle=True)
scores = np.load('scores.npy', allow_pickle=True)

# Visualize A Prediction

In [ ]:
rnd_idx = np.random.randint(0,len(data))
# visualize ground truth
data.visualize(rnd_idx)
plt.title('Ground Truth')

# visualize the predicted masks
data.visualize_prediction(rnd_idx, boxes[rnd_idx], masks[rnd_idx], labels[rnd_idx])
plt.title('Predicted Instances Raw')

# visualize the predicted masks after post-processing
data.visualize_prediction(rnd_idx, boxes[rnd_idx], masks[rnd_idx], labels[rnd_idx], unify=True)

In [ ]:
mIoU, mAP, overall_iou, processed_boxes, processed_labels, processed_masks, processed_scores, unified_masks = data.evaluate_precitions(boxes, labels, masks, scores, return_processed=True)